In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import math
import re
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import sklearn.metrics.pairwise

In [2]:
# read in the dataset
with open("train.dat", "r") as tr:
    train_data = tr.readlines()

In [11]:
print len(train_data[10].split())
print len(train_data[1].split())

print train_data[10]
print train_data[1]

212
380
718 1 20883 2 759 3 16 1 18923 1 2402 1 17 1 18925 1 18926 2 2281 1 4027 2 1435 1 9150 1 9033 1 1359 1 6284 1 1992 1 407 1 443 1 18210 5 562 1 166 2 924 1 169 7 926 1 18218 1 61 1 1126 1 1360 5 29 1 1046 2 3422 1 1244 2 66 2 1362 1 4073 1 2453 1 3506 1 1364 3 1283 1 69 1 1366 1 9160 1 3464 1 1367 4 2097 1 2579 1 1886 1 8 1 332 2 12205 1 2979 3 619 1 692 3 11438 1 577 1 30 1 11917 1 15276 1 3113 2 2105 1 1371 1 3038 2 9050 1 6063 1 78 1 1378 3 1496 1 142 1 421 3 540 2 264 1 428 1 384 2 10953 1 10638 4 946 1 785 2 21683 2 21684 1 21685 1 21686 1 10797 1 21687 1 20679 1 2033 1 21688 1 1026 2 18916 1 18917 1 989 1 18918 1 3045 4 2155 2 1382 1 2157 1 1149 3 4093 5 6911 2 117 1 2995 2 1789 3 154 1 634 2 393 1 555 1

358 3 759 2 1070 1 3414 2 2127 1 57 2 1633 1 2605 1 1075 1 2281 1 3416 1 1391 3 2680 1 3256 1 5670 1 5671 1 4547 2 5672 1 5673 1 204 1 5674 1 5675 1 5676 1 5677 1 523 1 5678 1 5679 1 362 1 763 4 844 1 682 2 369 2 289 1 61 1 1403 1 1242 1 3025 1 3109 1 2057 1 1409 1 5605 1

In [12]:
def csr_read(fname, ftype="csr", nidx=1):
    r""" 
        Read CSR matrix from a text file. 
        
        \param fname File name for CSR/CLU matrix
        \param ftype Input format. Acceptable formats are:
            - csr - Compressed sparse row
            - clu - Cluto format, i.e., CSR + header row with "nrows ncols nnz"
        \param nidx Indexing type in CSR file. What does numbering of feature IDs start with?
    """
    
    with open(fname) as f:
        lines = f.readlines()
    
    if ftype == "clu":
        p = lines[0].split()
        nrows = int(p[0])
        ncols = int(p[1])
        nnz = long(p[2])
        lines = lines[1:]
        assert(len(lines) == nrows)
    elif ftype == "csr":
        nrows = len(lines)
        ncols = 0 
        nnz = 0 
        for i in xrange(nrows):
            p = lines[i].split()
            if len(p) % 2 != 0:
                raise ValueError("Invalid CSR matrix. Row %d contains %d numbers." % (i, len(p)))
            nnz += len(p)/2
            for j in xrange(0, len(p), 2): 
                cid = int(p[j]) - nidx
                if cid+1 > ncols:
                    ncols = cid+1
    else:
        raise ValueError("Invalid sparse matrix ftype '%s'." % ftype)
    val = np.zeros(nnz, dtype=np.float)
    ind = np.zeros(nnz, dtype=np.int)
    ptr = np.zeros(nrows+1, dtype=np.long)
    n = 0 
    for i in xrange(nrows):
        p = lines[i].split()
        for j in xrange(0, len(p), 2): 
            ind[n] = int(p[j]) - nidx
            val[n] = float(p[j+1])
            n += 1
        ptr[i+1] = n 
    
    assert(n == nnz)
    
    return csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.float)

In [15]:
matrix= csr_read("train.dat")

In [ ]:
print matrix.toarray().shape
print matrix.todense().shape

M = matrix.todense()
distances = sklearn.metrics.pairwise.pairwise_distances(M,M)
print distances

(8580L, 126355L)
(8580L, 126355L)


In [21]:
mtx = csr_matrix((3, 4), dtype=np.int8)

In [32]:
print mtx.todense().shape


(3L, 3L)


In [31]:
data = np.array([1, 2, 3, 4, 5, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
indptr = np.array([0, 2, 3, 6])
mtx = csr_matrix((data, indices, indptr), shape=(3, 3))
print mtx
mtx.todense()


  (0, 0)	1
  (0, 2)	2
  (1, 2)	3
  (2, 0)	4
  (2, 1)	5
  (2, 2)	6


matrix([[1, 0, 2],
        [0, 0, 3],
        [4, 5, 6]])

In [2]:
UNCLASSIFIED = False
NOISE = None

def _dist(p,q):
	return math.sqrt(np.power(p-q,2).sum())

def _eps_neighborhood(p,q,eps):
	return _dist(p,q) < eps


#calculate distance to all points(columns) in the matrix and add to seed if it is < eps
def _region_query(m, point_id, eps):
    n_points = m.shape[1]
    seeds = []
    for i in range(0, n_points):
        if _eps_neighborhood(m[:,point_id], m[:,i], eps):
            seeds.append(i)
    return seeds

In [3]:
def _expand_cluster(m, classifications, point_id, cluster_id, eps, min_points):
    seeds = _region_query(m, point_id, eps)
    if len(seeds) < min_points:
        classifications[point_id] = NOISE
        return False
    else:
        classifications[point_id] = cluster_id
        for seed_id in seeds:
            classifications[seed_id] = cluster_id
            
        while len(seeds) > 0:
            current_point = seeds[0]
            results = _region_query(m, current_point, eps)
            if len(results) >= min_points:
                for i in range(0, len(results)):
                    result_point = results[i]
                    if classifications[result_point] == UNCLASSIFIED or \
                       classifications[result_point] == NOISE:
                        if classifications[result_point] == UNCLASSIFIED:
                            seeds.append(result_point)
                        classifications[result_point] = cluster_id
            seeds = seeds[1:]
        return True

In [4]:
def dbscan(m, eps, min_points):
    """Implementation of Density Based Spatial Clustering of Applications with Noise
    See https://en.wikipedia.org/wiki/DBSCAN
    
    scikit-learn probably has a better implementation
    
    Uses Euclidean Distance as the measure
    
    Inputs:
    m - A matrix whose columns are feature vectors(each column represents a record)
    eps - Maximum distance two points can be to be regionally related
    min_points - The minimum number of points to make a cluster
    
    Outputs:
    An array with either a cluster id number or dbscan.NOISE (None) for each
    column vector in m.
    """
    cluster_id = 1
    
    #this will have the number of records.
    n_points = m.shape[1] 
    print n_points
    
    #Assigning default classifications to all records.
    classifications = [UNCLASSIFIED] * n_points
    print classifications
    
    #loop over number of records
    for point_id in range(0, n_points):
        #get the current record
        point = m[:,point_id]
        print point.shape
        if classifications[point_id] == UNCLASSIFIED:
            if _expand_cluster(m, classifications, point_id, cluster_id, eps, min_points):
                cluster_id = cluster_id + 1
    return classifications

In [32]:
def test_dbscan():
    m = np.matrix('1 1.2 0.8 3.7 3.9 3.6 10; 1.1 0.8 1 4 3.9 4.1 10')
    eps = 0.5
    min_points = 2
    print "sudheer"
    print dbscan(m, eps, min_points)
    assert dbscan(m, eps, min_points) == [1, 1, 1, 2, 2, 2, None]

In [9]:
test_dbscan()

sudheer
[1, 1, 1, 2, 2, 2, None]


In [35]:
m = np.matrix('1 1.2 0.8 3.7 3.9 3.6 10; 1.1 0.8 1 4 3.9 4.1 10')

m = np.matrix('1 2 3 4 5 6 7 8 9;11 22 33 44 55 66 77 88 99')
eps = 0.5
min_points = 2
print "sudheer"
print m.shape[1]
print "sammeta"
print dbscan(m, eps, min_points)

sudheer
9
sammeta
9
[False, False, False, False, False, False, False, False, False]
(2L, 1L)
(2L, 1L)
(2L, 1L)
(2L, 1L)
(2L, 1L)
(2L, 1L)
(2L, 1L)
(2L, 1L)
(2L, 1L)
[None, None, None, None, None, None, None, None, None]


In [90]:
m = np.matrix('1 1.2 0.8 3.7 3.9 3.6 10; 1.1 0.8 1 4 3.9 4.1 10; 1.1 0.9 1 7 3.9 4.1 10')
m = np.matrix('1 1.2 0.8 3.7 3.9 3.6 10; 1.1 0.8 1 4 3.9 4.1 10').T
print m
rec_count = m.shape[0]


#Calculating distance between each pair of points
dist_matrix= np.zeros((rec_count,rec_count))
for row in range(0, rec_count):
    current =  m[row, :]
    #dist_matrix.append(distance_from_point(current, row, m, dist_matrix))
    distance_from_point(current, row, m, dist_matrix)

    
#print dist_matrix    

#Count core points
core_point_indexes= []
not_core_point_indexes = []
eps = 0.5
min_points = 2


#using index notation(code below is more readable.)
# for row in range(0, rec_count):
#     print dist_matrix[row]
#     print dist_matrix[row][np.where( dist_matrix[row] <= eps )].size
#     if dist_matrix[row][np.where( dist_matrix[row] <= eps )].size > min_points:
#         core_point_indexes.append(row)

# print core_point_indexes


#Identifying core and not core points.
row = 0
for rec in dist_matrix:
    #print rec[np.where( rec <= eps )].size
    if rec[np.where( rec <= eps )].size > min_points:
        core_point_indexes.append(row)
    else:
        not_core_point_indexes.append(row)
    row += 1

print core_point_indexes
print not_core_point_indexes



#Identifying border points, its closest core point and noise points
border_point_map = {}
noise_point_index= []

for i in not_core_point_indexes:
    min = 0
    for j in core_point_indexes:
        if dist_matrix[i][j] <= eps:
            if (min > 0 and dist_matrix[i][j] < min) or min == 0:
                border_point_map[i] = j
                min = dist_matrix[i][j]
    if min == 0:
        noise_point_index.append(i)

print noise_point_index


cluster= np.zeros(rec_count)
cluster_count = 0
for i in core_point_indexes:
    if cluster[i] == 0:
        cluster_count +=1
        cluster[i] = cluster_count
    for j in core_point_indexes:
        if cluster[j] == 0  and dist_matrix[i][j] <= eps:
            cluster[j] = cluster[i]
    
print cluster

#Clustering the core points

#Creating not oc
# border_point_indexes = []
# core_size = len(core_point_indexes)
# j=0
# for i in range(0, rec_count):
#         print "j is", j
#         if i < core_point_indexes[j]:
#             border_point_indexes.append(i)
#         elif i == core_point_indexes[j]:
#             continue
#         else:
#             if j+1 < core_size:
#                 j+=1
#                 if i < core_point_indexes[j]:
#                     border_point_indexes.append(i)
#             else:
#                 for a in range(core_point_indexes[j]+1, rec_count):
#                     border_point_indexes.append(a)
#                 break

print border_point_indexes            


[[  1.    1.1]
 [  1.2   0.8]
 [  0.8   1. ]
 [  3.7   4. ]
 [  3.9   3.9]
 [  3.6   4.1]
 [ 10.   10. ]]
[0, 1, 2, 3, 4, 5]
[6]
[6]
[ 1.  1.  1.  2.  2.  2.  0.]
[2]


In [10]:
#calculate distance to all points(columns) in the matrix and add to seed if it is < eps
#def distance_from_point(current, m):
#     total_records = m.shape[0]
#     distances = []
#     for i in range(0, total_records):
#         d = math.sqrt(np.power(current-m[i,:],2).sum())
#         distances.append(d)
#     return distances


def distance_from_point(current, row, m, dist_matrix):
    total_records = m.shape[0]
    for i in range(row+1, total_records):
        d = math.sqrt(np.power(current-m[i,:],2).sum())
        dist_matrix[row][i] = dist_matrix[i][row] = d
    

In [81]:
# x = np.arange(9.).reshape(3, 3)
# print x

# print np.where( x > 5 )

# m = np.matrix('1 2 3 4 5 6 10; 1 12 13 14 15 16 20; 21 22 23 24 25 26 30')

# print m[np.where( m[0] > 1.0 )].size               # Note: result is 1D.

# print np.where(x < 5, x, -1)               # Note: broadcasting.




#Count border points
rec_count=10
core_point_indexes = [0,9]
border_point_indexes = []
size = len(core_point_indexes)
j=0
for i in range(0, rec_count):
        print "j is", j
        if i < core_point_indexes[j]:
            print i , "sudheer"
            border_point_indexes.append(i)
        elif i == core_point_indexes[j]:
            continue
        else:
            if j+1 < size:
                j+=1
                if i < core_point_indexes[j]:
                    border_point_indexes.append(i)
            else:
                for a in range(core_point_indexes[j]+1, rec_count):
                    border_point_indexes.append(a)
                break

print border_point_indexes            

j is 0
j is 0
j is 1
2 sudheer
j is 1
3 sudheer
j is 1
4 sudheer
j is 1
5 sudheer
j is 1
6 sudheer
j is 1
7 sudheer
j is 1
8 sudheer
j is 1
[1, 2, 3, 4, 5, 6, 7, 8]
